# Imports

In [1]:
import pyodbc
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.frequent_patterns import apriori, association_rules
import plotly.express as px
import plotly.graph_objects as go

warnings.simplefilter("ignore")

# Functions

## Database

In [2]:
def get_tables_db(cursor:pyodbc.Cursor)->list:

    names = []
    for row in cursor.tables():
        if "MSys" not in str(row.table_name):
            names.append(row.table_name)
    
    return names
    
def get_data(table_name:str, conn:pyodbc.connect)->pd.DataFrame:
    
    return pd.read_sql_query("select * from "+table_name, con=conn)

## Preprocessing

In [3]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
categorics = ['category', 'object']

def show_stats_numeric(df:pd.DataFrame):

    columns = df.select_dtypes(include=numerics).columns

    for column in columns:
        print(f"col:{column}\nnulls:{df[column].isnull().sum()}, duplicated:{df[column].duplicated().sum()}, nunique:{df[column].nunique()}, values_uniques:{sorted(df[column].unique())}, value_min:{df[column].min()}, value_max:{df[column].max()} \n")

def show_stats_categorical(df:pd.DataFrame):

    columns = df.select_dtypes(include=categorics).columns

    for column in columns:
        print(f"col:{column}\n{df[column].value_counts()}")

## Machine Learning

In [4]:
def generate_association_rules(df:pd.DataFrame, min_sup:float, metric_use:str, threshold:float  )->pd.DataFrame:

    frequent_itens = apriori(df, min_support=min_sup, use_colnames=True)

    rules = association_rules(frequent_itens, metric=metric_use, min_threshold=threshold)
    rules = rules.query("lift >= 1").sort_values(by=['confidence','support'], ascending = [False, False])
   
    return frequent_itens, rules

def format_data_to_apriori(df:pd.DataFrame, columns:list)->pd.DataFrame:

    df_apriori = df[columns]
    df_apriori = pd.get_dummies(df_apriori)
    df_apriori = df_apriori.groupby(columns[0]).sum()
    
    for column in df_apriori.columns:
        df_apriori.loc[ df_apriori[column] > 1 , column] = 1

    return df_apriori

def calculate_potential_transactions_in_rules(df_rules:pd.DataFrame, df_description:pd.DataFrame)->pd.DataFrame:

    result = [] 

    for row in df_rules.itertuples():

        a = row.antecedents
        c = row.consequents
        
        true_ant = df_description[ df_description[a] == 1].dropna(subset=a).index.to_list()
        false_con = df_description[df_description[c] == 0].dropna(subset=c).index.to_list()

        potencial = df_description[df_description.index.isin(true_ant) & df_description.index.isin(false_con)].index.nunique()
        tot_ant = df_description[df_description.index.isin(true_ant)].index.nunique()

        result.append({'antecedents':a, 'consequents':c, 'potencial_transactions':potencial, 'total_antecedents': tot_ant })

    return pd.DataFrame(result)

def treat_string(values:str)->str:

    values = values.replace("[", "{")
    values = values.replace("]", "}")
    values = values.replace("descrição_", "")
    values = values.replace("'", "")
    
    return values

# Table Review

In [5]:
db_path = 'C:/Users/annap/Documentos/codigos/teste_pratico_sicredi/data/compras2014.mdb'
driver = 'Microsoft Access Driver (*.mdb, *.accdb)'

conn = pyodbc.connect(f"Driver={driver};DBQ={db_path};")
cursor = conn.cursor()

table_names = get_tables_db(cursor)
table_names

['itemtransacao', 'itens', 'transacoes', 'itens Consulta']

## Itens

Contém os itens disponíveis, sua descrição marca e tipo

Total de itens únicos: 10

In [6]:
df_itens = get_data('itens', conn)
print(f"Shape:{df_itens.shape}\nDuplicated lines:{df_itens.duplicated().sum()}")
df_itens.head()

Shape:(10, 4)
Duplicated lines:0


,codItem,descrição,marca,tipo
0,1,banana prata,banana,fruta
1,2,banana caturra,banana,fruta
2,3,limao Taiti,limao,fruta
3,4,limão siciliano,limão,fruta
4,5,coca,coca,refrigerante


In [7]:
show_stats_numeric(df_itens)

col:codItem
nulls:0, duplicated:0, nunique:10, values_uniques:[1, 2, 3, 4, 5, 6, 7, 10, 11, 12], value_min:1, value_max:12 



In [8]:
show_stats_categorical(df_itens)

col:descrição
banana prata        1
banana caturra      1
limao Taiti         1
limão siciliano     1
coca                1
coca light          1
coca lemon light    1
omo collors         1
omo progress        1
ariel total         1
Name: descrição, dtype: int64
col:marca
coca      3
banana    2
omo       2
limao     1
limão     1
ariel     1
Name: marca, dtype: int64
col:tipo
fruta           4
sabao em po     3
refrigerante    2
refirgerante    1
Name: tipo, dtype: int64


In [9]:
df_itens[ df_itens['descrição'].str.contains("lim") | df_itens['marca'].str.contains("lim") ]

,codItem,descrição,marca,tipo
2,3,limao Taiti,limao,fruta
3,4,limão siciliano,limão,fruta


In [10]:
df_itens[ df_itens['tipo'].str.contains("ref") ]

,codItem,descrição,marca,tipo
4,5,coca,coca,refrigerante
5,6,coca light,coca,refrigerante
6,7,coca lemon light,coca,refirgerante


## Transactions

Contém o valor total e o tipo de pagamento.

Representa o volume de transações feitas: 34


In [11]:
df_transactions = get_data('transacoes', conn)
print(f"Shape:{df_transactions.shape}\nDuplicated lines:{df_transactions.duplicated().sum()}")
df_transactions.head()

Shape:(34, 3)
Duplicated lines:0


,IDTransação,valorTotal,tipo pagamento
0,1,15,ch
1,2,20,ch
2,3,14,es
3,4,19,ch
4,5,15,es


In [12]:
show_stats_numeric(df_transactions)
show_stats_categorical(df_transactions)

col:IDTransação
nulls:0, duplicated:0, nunique:34, values_uniques:[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35], value_min:1, value_max:35 

col:valorTotal
nulls:0, duplicated:21, nunique:13, values_uniques:[10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 24, 30], value_min:10, value_max:30 

col:tipo pagamento
es    17
ch    10
cc     7
Name: tipo pagamento, dtype: int64


## Item Transaction

Contém o id da transação e os itens de cada transação.

Transações únicas e ids únicos: (27, 10)

In [13]:
df_item_transaction = get_data('itemtransacao', conn)
print(f"Shape:{df_item_transaction.shape}\nDuplicated lines:{df_item_transaction.duplicated().sum()}")
df_item_transaction.head()

Shape:(85, 2)
Duplicated lines:0


,IDTransação,item
0,14,1
1,14,3
2,14,6
3,15,1
4,15,3


In [14]:
show_stats_numeric(df_item_transaction)
show_stats_categorical(df_item_transaction)

col:IDTransação
nulls:0, duplicated:58, nunique:27, values_uniques:[1, 2, 3, 5, 6, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35], value_min:1, value_max:35 

col:item
nulls:0, duplicated:75, nunique:10, values_uniques:[1, 2, 3, 4, 5, 6, 7, 10, 11, 12], value_min:1, value_max:12 



In [15]:
aux_supp = df_item_transaction.groupby("item")['IDTransação'].count().sort_values(ascending=False).reset_index(name='quantidade_de_saida')
aux_supp['porcentagem'] = ( aux_supp['quantidade_de_saida'] / df_item_transaction.IDTransação.nunique() ) * 100
aux_supp

,item,quantidade_de_saida,porcentagem
0,1,12,44.444444
1,6,12,44.444444
2,2,10,37.037037
3,3,9,33.333333
4,4,9,33.333333
5,10,8,29.629630
6,12,8,29.629630
7,7,6,22.222222
8,11,6,22.222222
9,5,5,18.518519


In [16]:
aux_supp.quantile(q=0.5, interpolation='linear', method='single')['porcentagem']

31.481481481481477

## Consultas

In [17]:
df_consultas = get_data('itens Consulta', conn)
print(f"Shape:{df_consultas.shape}\nDuplicated lines:{df_consultas.duplicated().sum()}")
df_consultas.head()

Shape:(10, 4)
Duplicated lines:0


,codItem,descrição,marca,tipo
0,1,banana prata,banana,fruta
1,2,banana caturra,banana,fruta
2,3,limao Taiti,limao,fruta
3,4,limão siciliano,limão,fruta
4,5,coca,coca,refrigerante


In [18]:
show_stats_numeric(df_consultas)
show_stats_categorical(df_consultas)

col:codItem
nulls:0, duplicated:0, nunique:10, values_uniques:[1, 2, 3, 4, 5, 6, 7, 10, 11, 12], value_min:1, value_max:12 

col:descrição
banana prata        1
banana caturra      1
limao Taiti         1
limão siciliano     1
coca                1
coca light          1
coca lemon light    1
omo collors         1
omo progress        1
ariel total         1
Name: descrição, dtype: int64
col:marca
coca      3
banana    2
omo       2
limao     1
limão     1
ariel     1
Name: marca, dtype: int64
col:tipo
fruta           4
sabao em po     3
refrigerante    2
refirgerante    1
Name: tipo, dtype: int64


## Valid Consultas e Itens

O objetivo é validar se a tabela itens e consultas são iguais

In [19]:
assert df_consultas.codItem.nunique() == df_itens.codItem.nunique(), "A quantidade de intens únicos nas tabelas são diferentes."

In [20]:
df_consultas[ df_consultas.codItem.isin(df_itens.codItem.unique())]

,codItem,descrição,marca,tipo
0,1,banana prata,banana,fruta
1,2,banana caturra,banana,fruta
2,3,limao Taiti,limao,fruta
3,4,limão siciliano,limão,fruta
4,5,coca,coca,refrigerante
5,6,coca light,coca,refrigerante
6,7,coca lemon light,coca,refirgerante
7,10,omo collors,omo,sabao em po
8,11,omo progress,omo,sabao em po
9,12,ariel total,ariel,sabao em po


In [21]:
df_itens[ df_itens.codItem.isin(df_consultas.codItem.unique())]

,codItem,descrição,marca,tipo
0,1,banana prata,banana,fruta
1,2,banana caturra,banana,fruta
2,3,limao Taiti,limao,fruta
3,4,limão siciliano,limão,fruta
4,5,coca,coca,refrigerante
5,6,coca light,coca,refrigerante
6,7,coca lemon light,coca,refirgerante
7,10,omo collors,omo,sabao em po
8,11,omo progress,omo,sabao em po
9,12,ariel total,ariel,sabao em po


In [22]:
valid = pd.merge(df_itens, df_consultas, on='codItem', suffixes=['_it','_con'])
valid

,codItem,descrição_it,marca_it,tipo_it,descrição_con,marca_con,tipo_con
0,1,banana prata,banana,fruta,banana prata,banana,fruta
1,2,banana caturra,banana,fruta,banana caturra,banana,fruta
2,3,limao Taiti,limao,fruta,limao Taiti,limao,fruta
3,4,limão siciliano,limão,fruta,limão siciliano,limão,fruta
4,5,coca,coca,refrigerante,coca,coca,refrigerante
5,6,coca light,coca,refrigerante,coca light,coca,refrigerante
6,7,coca lemon light,coca,refirgerante,coca lemon light,coca,refirgerante
7,10,omo collors,omo,sabao em po,omo collors,omo,sabao em po
8,11,omo progress,omo,sabao em po,omo progress,omo,sabao em po
9,12,ariel total,ariel,sabao em po,ariel total,ariel,sabao em po


In [23]:
valid[valid['descrição_con'] != valid['descrição_it']]

,codItem,descrição_it,marca_it,tipo_it,descrição_con,marca_con,tipo_con


In [24]:
valid[valid['marca_con'] != valid['marca_it']]

,codItem,descrição_it,marca_it,tipo_it,descrição_con,marca_con,tipo_con


In [25]:
valid[valid['tipo_con'] != valid['tipo_it']]

,codItem,descrição_it,marca_it,tipo_it,descrição_con,marca_con,tipo_con


# Construct Fact Table

A tabela fato é onde concentrará a representação das vendas contendo todas as informações de protuto, pagamento e transação

### Validando a tabela de transações e a tabela de item transação

In [26]:
df_item_transaction[ df_item_transaction.IDTransação.isin(df_transactions.IDTransação)]['IDTransação'].nunique()

27

In [27]:
df_transactions[ ~df_transactions.IDTransação.isin(df_item_transaction.IDTransação.unique())]['IDTransação'].nunique()

7

In [28]:
df_transactions[ ~df_transactions.IDTransação.isin(df_item_transaction.IDTransação.unique())]['IDTransação'].unique()

array([ 4,  7,  8,  9, 10, 11, 12], dtype=int64)

In [29]:
assert df_item_transaction[ df_item_transaction.item.isin(df_itens.codItem)]['item'].nunique() == df_itens[ df_itens.codItem.isin(df_item_transaction.item)]['codItem'].nunique(), "A quantidade de itens únicos não batem."

### Merges

In [30]:
df_fact = pd.merge(df_item_transaction, df_transactions, on='IDTransação', how='inner')
df_fact = pd.merge(df_fact, df_itens, left_on='item', right_on='codItem', how='inner')

print(f"Shape:{df_fact.shape}\nDuplicated lines:{df_fact.duplicated().sum()}")

df_fact = df_fact.rename(columns={'item':'idItem', 'IDTransação':'idTransação'})
df_fact = df_fact[['idTransação', 'idItem', 'marca','descrição', 'tipo', 'valorTotal', 'tipo pagamento']]
df_fact = df_fact.sort_values(by=['idTransação', 'idItem'], ascending=True)

df_fact.head()

Shape:(85, 8)
Duplicated lines:0


,idTransação,idItem,marca,descrição,tipo,valorTotal,tipo pagamento
10,1,1,banana,banana prata,fruta,15,ch
65,1,4,limão,limão siciliano,fruta,15,ch
37,1,12,ariel,ariel total,sabao em po,15,ch
38,2,12,ariel,ariel total,sabao em po,20,ch
55,3,2,banana,banana caturra,fruta,14,es


## Data Exploration Fact Table

### Produto

In [31]:
product = df_fact.groupby("descrição")['idItem'].count().sort_values(ascending=False).reset_index(name='quantidade')
product

,descrição,quantidade
0,banana prata,12
1,coca light,12
2,banana caturra,10
3,limao Taiti,9
4,limão siciliano,9
5,ariel total,8
6,omo collors,8
7,coca lemon light,6
8,omo progress,6
9,coca,5


In [32]:
colors=['#00a200','#19ab19', '#32b432', '#4cbd4c', '#66c766', '#7fd07f', '#99d999', '#b2e3b2', '#cceccc', '#e5f5e5']

fig = px.bar( product, y="descrição", x="quantidade", color="descrição", orientation="h", color_discrete_sequence=colors, 
              title='Volume de Transações por Item',  text_auto=True, 
            )

fig.update_traces(  textfont_size=16, textangle=0,textposition="outside") 
fig.update_layout(  showlegend=False, plot_bgcolor='rgba(0, 0, 0, 0)', font=dict( family="Arial, monospace", size=18),
                    yaxis_title="")  
fig.update_xaxes(visible=False)
fig.show()

### Tipo

In [33]:
tipo = df_fact.tipo.value_counts().reset_index(name='quantidade').rename(columns={'index':'tipo'})
tipo

,tipo,quantidade
0,fruta,40
1,sabao em po,22
2,refrigerante,17
3,refirgerante,6


In [34]:
fig = go.Figure(data=[go.Pie(labels=tipo.tipo, values=tipo.quantidade, hole=.3, marker_colors=colors, 
                textfont_size=14, textinfo='label+percent')])
fig.update_layout( width=700, height=500, title='Proporção de Transações por Tipo')
fig.show()

### Método de pagamento

In [35]:
df_fact.groupby("tipo pagamento")['idTransação'].nunique()

tipo pagamento
cc     4
ch     8
es    15
Name: idTransação, dtype: int64

In [36]:
df_fact.drop_duplicates(subset='idTransação').groupby("tipo pagamento")['valorTotal'].sum()

tipo pagamento
cc     74
ch    161
es    213
Name: valorTotal, dtype: int64

# Define Association Rules

In [37]:
df_description = format_data_to_apriori(df_fact, ['idTransação', 'descrição'])
df_description.head()

,descrição_ariel total,descrição_banana caturra,descrição_banana prata,descrição_coca,descrição_coca lemon light,descrição_coca light,descrição_limao Taiti,descrição_limão siciliano,descrição_omo collors,descrição_omo progress
idTransação,,,,,,,,,,
1,1,0,1,0,0,0,0,1,0,0
2,1,0,0,0,0,0,0,0,0,0
3,1,1,0,0,0,0,0,0,0,0
5,0,1,0,1,0,0,0,0,0,0
6,1,0,1,0,0,0,1,0,0,0


In [38]:
assert df_description.shape[0] == df_fact.idTransação.nunique(), "Erro ao transformar as transações para entrada do algoritmo."

In [39]:
frequency_description, rules_description = generate_association_rules(df_description, min_sup=0.1, metric_use='confidence', threshold=0.5)
rules_description.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
8,"(descrição_coca light, descrição_banana caturra)",(descrição_limão siciliano),0.148148,0.333333,0.148148,1.000000,3.000,0.098765,inf
9,"(descrição_coca light, descrição_limão siciliano)",(descrição_banana caturra),0.148148,0.370370,0.148148,1.000000,2.700,0.093278,inf
10,"(descrição_banana caturra, descrição_limão sic...",(descrição_coca light),0.148148,0.444444,0.148148,1.000000,2.250,0.082305,inf
16,"(descrição_omo progress, descrição_limao Taiti)",(descrição_coca light),0.111111,0.444444,0.111111,1.000000,2.250,0.061728,inf
6,(descrição_omo progress),(descrição_coca light),0.222222,0.444444,0.185185,0.833333,1.875,0.086420,3.333333


In [40]:
frequency_description['itemsets'] = frequency_description.itemsets.apply(lambda x: list(x))
frequency_description['lenght'] = frequency_description.itemsets.apply(lambda x : len(x))
aux = frequency_description.query("lenght == 1")
aux


,support,itemsets,lenght
0,0.296296,[descrição_ariel total],1
1,0.370370,[descrição_banana caturra],1
2,0.444444,[descrição_banana prata],1
3,0.185185,[descrição_coca],1
4,0.222222,[descrição_coca lemon light],1
5,0.444444,[descrição_coca light],1
6,0.333333,[descrição_limao Taiti],1
7,0.333333,[descrição_limão siciliano],1
8,0.296296,[descrição_omo collors],1
9,0.222222,[descrição_omo progress],1


In [41]:
df_fact[df_fact['descrição'] == "coca"]

,idTransação,idItem,marca,descrição,tipo,valorTotal,tipo pagamento
70,5,5,coca,coca,refrigerante,15,es
66,18,5,coca,coca,refrigerante,12,es
67,21,5,coca,coca,refrigerante,12,es
68,29,5,coca,coca,refrigerante,12,es
69,32,5,coca,coca,refrigerante,30,ch


In [42]:
format_slide = rules_description[['antecedents', 'consequents', 'support', 'confidence']].copy()

format_slide['antecedents'] = format_slide.antecedents.apply(lambda x: list(x))
format_slide['consequents'] = format_slide.consequents.apply(lambda x: list(x))

format_slide = format_slide.astype({'antecedents':str, 'consequents':str})

format_slide.antecedents = format_slide.antecedents.apply( lambda x: treat_string(x) )
format_slide.consequents = format_slide.consequents.apply( lambda x: treat_string(x) )

format_slide['rules'] = format_slide['antecedents'] + " -> " + format_slide['consequents']

format_slide[['rules', 'support', 'confidence']]

,rules,support,confidence
8,"{coca light, banana caturra} -> {limão siciliano}",0.148148,1.000000
9,"{coca light, limão siciliano} -> {banana caturra}",0.148148,1.000000
10,"{banana caturra, limão siciliano} -> {coca light}",0.148148,1.000000
16,"{omo progress, limao Taiti} -> {coca light}",0.111111,1.000000
6,{omo progress} -> {coca light},0.185185,0.833333
12,"{banana prata, limao Taiti} -> {coca light}",0.111111,0.750000
13,"{coca light, limao Taiti} -> {banana prata}",0.111111,0.750000
15,"{coca light, limao Taiti} -> {omo progress}",0.111111,0.750000
1,{coca} -> {banana caturra},0.111111,0.600000
11,"{banana prata, coca light} -> {limao Taiti}",0.111111,0.600000


# Estimated Transactions: if rules applied

In [43]:
rules_to_estimate = rules_description.query("confidence != 1")

rules_to_estimate = rules_to_estimate[['antecedents', 'consequents']] 

rules_to_estimate['antecedents'] = rules_to_estimate.antecedents.apply(lambda x: list(x))
rules_to_estimate['consequents'] = rules_to_estimate.consequents.apply(lambda x: list(x))

result_potencial = calculate_potential_transactions_in_rules(rules_to_estimate, df_description)
result_potencial = result_potencial.sort_values(by='potencial_transactions', ascending=False)
result_potencial

,antecedents,consequents,potencial_transactions,total_antecedents
7,[descrição_limão siciliano],[descrição_banana prata],4,9
8,[descrição_ariel total],[descrição_limao Taiti],4,8
9,[descrição_omo collors],[descrição_banana prata],4,8
10,[descrição_omo collors],[descrição_coca light],4,8
11,[descrição_coca lemon light],[descrição_limao Taiti],3,6
12,[descrição_omo progress],[descrição_limao Taiti],3,6
4,[descrição_coca],[descrição_banana caturra],2,5
5,"[descrição_banana prata, descrição_coca light]",[descrição_limao Taiti],2,5
6,"[descrição_coca light, descrição_omo progress]",[descrição_limao Taiti],2,5
0,[descrição_omo progress],[descrição_coca light],1,6


In [44]:
result_potencial['taxa_de_aumento'] = (result_potencial['potencial_transactions'] * 100) / result_potencial['total_antecedents'] 
result_potencial

,antecedents,consequents,potencial_transactions,total_antecedents,taxa_de_aumento
7,[descrição_limão siciliano],[descrição_banana prata],4,9,44.444444
8,[descrição_ariel total],[descrição_limao Taiti],4,8,50.000000
9,[descrição_omo collors],[descrição_banana prata],4,8,50.000000
10,[descrição_omo collors],[descrição_coca light],4,8,50.000000
11,[descrição_coca lemon light],[descrição_limao Taiti],3,6,50.000000
12,[descrição_omo progress],[descrição_limao Taiti],3,6,50.000000
4,[descrição_coca],[descrição_banana caturra],2,5,40.000000
5,"[descrição_banana prata, descrição_coca light]",[descrição_limao Taiti],2,5,40.000000
6,"[descrição_coca light, descrição_omo progress]",[descrição_limao Taiti],2,5,40.000000
0,[descrição_omo progress],[descrição_coca light],1,6,16.666667


In [45]:
result_potencial = result_potencial.astype({'antecedents':str, 'consequents':str})

result_potencial.antecedents = result_potencial.antecedents.apply( lambda x: treat_string(x) )
result_potencial.consequents = result_potencial.consequents.apply( lambda x: treat_string(x) )

result_potencial['rules'] = result_potencial['antecedents'] + " -> " + result_potencial['consequents']
result_potencial[['rules', 'potencial_transactions', 'total_antecedents', 'taxa_de_aumento' ]]

,rules,potencial_transactions,total_antecedents,taxa_de_aumento
7,{limão siciliano} -> {banana prata},4,9,44.444444
8,{ariel total} -> {limao Taiti},4,8,50.000000
9,{omo collors} -> {banana prata},4,8,50.000000
10,{omo collors} -> {coca light},4,8,50.000000
11,{coca lemon light} -> {limao Taiti},3,6,50.000000
12,{omo progress} -> {limao Taiti},3,6,50.000000
4,{coca} -> {banana caturra},2,5,40.000000
5,"{banana prata, coca light} -> {limao Taiti}",2,5,40.000000
6,"{coca light, omo progress} -> {limao Taiti}",2,5,40.000000
0,{omo progress} -> {coca light},1,6,16.666667
